In [8]:
import argparse
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import glob

from dcase_models.model.container import *
from dcase_models.data.data_generator import DataGenerator
from dcase_models.data.features import MelSpectrogram
from dcase_models.data.datasets import UrbanSound8k
from dcase_models.util.files import load_json, mkdir_if_not_exists, save_pickle, load_pickle
from dcase_models.util.data import evaluation_setup

sys.path.append('../')
from apnet.model import *
from apnet.layers import PrototypeLayer, WeightedSum
from apnet.datasets import GoogleSpeechCommands, MedleySolosDb

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
datasets_dict = {
    'UrbanSound8k': UrbanSound8k,
    'MedleySolosDb': MedleySolosDb,
    'GoogleSpeechCommands': GoogleSpeechCommands
}

In [5]:
model_name = 'APNet'
features_name = 'MelSpectrogram'

In [9]:
for dataset_name in datasets_dict.keys():

    # Get parameters
    exp_path = '../experiments'
    model_folder = os.path.join(exp_path, dataset_name, model_name)
    parameters_file = os.path.join(model_folder, 'config.json')
    params = load_json(parameters_file)
    params_dataset = params['datasets'][dataset_name]
    params_features = params['features'][features_name]
    params_model = params['models'][model_name]
    
    dataset_path = os.path.join(exp_path, params_dataset['dataset_path'])
    dataset = datasets_dict[dataset_name](dataset_path)
    
    features = MelSpectrogram(**params_features)
    features.check_if_extracted(dataset)
    
    mse = 0
    n = 0

    fold_list = dataset.fold_list if dataset_name == 'UrbanSound8k' else ['test']

    for fold_name in fold_list:
        exp_folder = os.path.join(model_folder, fold_name)
        scaler = load_pickle(os.path.join(exp_folder, 'scaler.pickle'))

        model_container = APNet(
            model=None, model_path=exp_folder, 
            custom_objects={
                'PrototypeLayer': PrototypeLayer,
                'WeightedSum': WeightedSum
            }
        )
        model_container.load_model_weights(exp_folder)

        data_gen = DataGenerator(
            dataset, features, folds=[fold_name],
            batch_size=params['train']['batch_size'],
            shuffle=False, train=True, scaler=scaler
        )

        for batch in range(len(data_gen)):
            X, Y = data_gen.get_data_batch(batch)

            reconstructions = model_container.model.predict(X)[1]

            X = scaler.inverse_transform(X)
            reconstructions = scaler.inverse_transform(reconstructions)
            
            mse += np.mean((reconstructions-X)**2)
            n += 1

    print(dataset_name, np.sqrt(mse/n))

ValueError: The supplied matrices should be of shape (nsrc, nsampl) but reference_sources.shape[0] = 256 and estimated_sources.shape[0] = 256 which is greater than mir_eval.separation.MAX_SOURCES = 100.  To override this check, set mir_eval.separation.MAX_SOURCES to a larger value.